# **Fine-Tune Whisper For Luganda ASR with 🤗 Transformers**

Adapted from [Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-whisper)

## Prepare Environment

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [7]:
%%capture
!pip install datasets==2.17.0
!pip install transformers==4.48.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer # jiwer is used for evaluation using WER and CER
!pip install accelerate -U # Restart runtime after running this cell
!pip install wandb
!pip install soundfile
!pip install librosa
!pip install matplotlib
!pip install evaluate
!pip install seaborn
!pip install pyctcdecode 
!pip install kenlm

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [ ]:
from datasets import load_dataset, Audio
import numpy as np

In [ ]:
# You need to log in on HuggingFace and accept the terms and conditions of the Mozilla Foundation common voice dataset
lg_cv_train = load_dataset("mozilla-foundation/common_voice_7_0", "lg", split="train", trust_remote_code=True)
lg_cv_valid = load_dataset("mozilla-foundation/common_voice_7_0", "lg", split="validation", trust_remote_code=True)
lg_cv_test = load_dataset("mozilla-foundation/common_voice_7_0", "lg", split="test", trust_remote_code=True)

In [ ]:
# Print the dataset to view a summary of the dataset
print(lg_cv_train)
print(lg_cv_valid)
print(lg_cv_test)

In [ ]:
# Select a sample from the dataset and view its audio colummn
lg_cv_train[0]['audio']

In [ ]:
lg_cv_train = lg_cv_train.select_columns(['audio', 'Transcriptions', 'duration'])
lg_cv_valid  = lg_cv_valid.select_columns(['audio', 'Transcriptions', 'duration'])
lg_cv_test  = lg_cv_test.select_columns(['audio', 'Transcriptions', 'duration'])

In [ ]:
# Use this code to compute number of samples required to make up a total duration of interest
cumsum_length = np.cumsum(lg_cv_train['duration'])
cumsum_length[cumsum_length < 18_000].shape

### Display samples from the dataset

In [83]:
# Show samples from the dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples= 10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = (random.randint(0, len(dataset)-1))
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(lg_cv_train.remove_columns(['audio']))

In [85]:
# normalize the transcripts. We are not training an orthographic model
def normalize(batch):
    batch['transcription'] = batch['Transcriptions'].lower()
    return batch

lg_cv_train = lg_cv_train.map(normalize)
lg_cv_valid = lg_cv_valid.map(normalize)
lg_cv_test = lg_cv_test.map(normalize)

In [22]:
# Obtain the vocabulary from the dataset
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
luganda_train_vocab   = lg_cv_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lg_cv_train.column_names)
luganda_valid_vocab   = lg_cv_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lg_cv_valid.column_names)

In [25]:
vocab_list = list(set(luganda_train_vocab["vocab"][0]) | set(luganda_valid_vocab["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

In some cases when the alphabet of the dataset differs from the alphabet of Luganda, we might have to clean the data by replaing some of the characters in the dataset with the closest character in the Luganda alphabet.

## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages:

1. A feature extractor which pre-processes the raw audio-inputs
2. The model which performs the sequence-to-sequence mapping
3. A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [27]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Luganda, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to
specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences:

In [1]:
from whisper_tokenizer import LugandaWhisperTokenizer

tokenizer = LugandaWhisperTokenizer.from_pretrained("openai/whisper-small", language = "lg", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'WhisperTokenizer'. 
The class this function is called from is 'LugandaWhisperTokenizer'.


### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [29]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(feature_extractor = feature_extractor, tokenizer = tokenizer)

### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [30]:
lg_cv_train = lg_cv_train.cast_column('audio', Audio(16000))
lg_cv_valid = lg_cv_valid.cast_column('audio', Audio(16000))

In [31]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch['length'] = batch['duration']

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

Note: This process may a take a long time to complete

In [ ]:
lg_cv_train = lg_cv_train.map(prepare_dataset, num_proc=1, remove_columns=lg_cv_train.column_names)
lg_cv_valid = lg_cv_valid.map(prepare_dataset, num_proc=1, remove_columns=lg_cv_valid.column_names)

In [ ]:
# Check the total duration of the dataset
print(f"Training dataset: {sum(lg_cv_train['length'])/3600} hours")
print(f"Validation dataset: {sum(lg_cv_train['length'])/3600} hours")

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) and the [character error rate (WER)](https://huggingface.co/spaces/evaluate-metric/cer) metric. We need to define a `compute_metrics` function that handles this computation.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it to transcribe speech in Luganda.

### Load a Pre-Trained Checkpoint

We'll start our fine-tuning run from the pre-trained Whisper `small` checkpoint,
the weights for which we need to load from the Hugging Face Hub. Again, this
is trivial through use of 🤗 Transformers!

In [35]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

We can disable the automatic language detection task performed during inference, and force the model to generate in Luganda. To do so, we set the [langauge](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.language)
and [task](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.task)
arguments to the generation config. We'll also set any [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)
to None, since this was the legacy way of setting the language and
task arguments:

In [37]:
model.generation_config.language = "Luganda"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [38]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [39]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

We'll use the word error rate (WER) and character error rate (CER) metric, the 'de-facto' metric for assessing
ASR systems. We'll load the WER and CER metric from 🤗 Evaluate:

In [40]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER and CER between the predictions and reference labels:

In [41]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    cer = 100 * cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [ ]:
# Wandb
import wandb

wandb.login()

In [ ]:
# Wandb arguments
%env WANDB_LOG_MODEL=end
%env WANDB_PROJECT=Luganda-ASR
%env WANDB_WATCH=all
%env WANDB_SILENT=true

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [44]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./repo",  # change to a repo name of your choice
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_ratio=0.1,
    group_by_length=True,
    num_train_epochs=50,
    gradient_checkpointing=True,
    bf16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    report_to=["wandb"],
    run_name="run-name",
    hub_private_repo = True,
    torch_compile=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    dataloader_num_workers=16,
    dataloader_pin_memory=True,
    dataloader_prefetch_factor=4,
    push_to_hub=True,
    save_total_limit=2,
    hub_model_id='username/repo', # replace this with your username and repo name
)

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [45]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=lg_cv_train,
    eval_dataset=lg_cv_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)

We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [ ]:
processor.save_pretrained(training_args.output_dir)

### Training

Training will take approximately 5-10 hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB.
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [47]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train(resume_from_checkpoint=False)
wandb.finish()
trainer.push_to_hub()

### Test the Model

In [ ]:
print(lg_cv_test)

In [50]:
import torch

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm

# Load the model from the HuggingFace Hub
pipe = pipeline("automatic-speech-recognition",model="username/repo", device =device, return_timestamps=True) 

In [ ]:
# Use this code to run inference on the test data
# Note: This requires the raw audio without any preprocessing
predictions = [prediction['text'] for prediction in tqdm(pipe(KeyDataset(lg_cv_test, "audio"), batch_size=4),\
                                                              desc= "Running inference", total= lg_cv_test.num_rows)]

In [87]:
lg_cv_test = lg_cv_test.add_column("predicted_string", predictions)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=lg_cv_test["predicted_string"], references=lg_cv_test["transcription"])))

In [ ]:
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=lg_cv_test["predicted_string"], references=lg_cv_test["transcription"])))

In [90]:
luganda_test_df = lg_cv_test.remove_columns(['audio', 'File No.', 'ENVIRONMENT', 'YEAR', 'SPEAKER_ID', 'Transcriptions']).to_pandas()

In [91]:
# Calculate the wer and cer for each sample in the test dataset
luganda_test_df["WER"] = luganda_test_df.apply(lambda row: wer_metric.compute(references= [row['transcription']], predictions=[row['predicted_string']]), axis=1)
luganda_test_df["CER"] = luganda_test_df.apply(lambda row: cer_metric.compute(references= [row['transcription']], predictions=[row['predicted_string']]), axis=1)

In [92]:
# save the predictions to a csv file for further analysis
luganda_test_df.to_csv("predictions/repo.csv", index= False)

In [ ]:
luganda_test_df.head()